In [1]:
import pandas as pd

In [2]:
nationaldex = pd.read_csv("data/pokedex_8gen(Update_04.21).csv")
# nationaldex = nationaldex[['name','type_1','type_2','ability_1','ability_2','ability_hidden','hp','attack','defense','sp_attack','sp_defense','speed']]
nationaldex = nationaldex[['name','ability_1','ability_2','ability_hidden','hp','attack','defense','sp_attack','sp_defense','speed']]

In [3]:
nationaldex["Species"] = nationaldex["name"].apply(lambda name: name.split()[0].lower())
nationaldex["Form"] = nationaldex["name"].apply(lambda name: name.split()[1].lower() if len(name.split()) > 1 else "base")
nationaldex.head()

,name,ability_1,ability_2,ability_hidden,hp,attack,defense,sp_attack,sp_defense,speed,Species,Form
0,Bulbasaur,Overgrow,NaN,Chlorophyll,45,49,49,65,65,45,bulbasaur,base
1,Ivysaur,Overgrow,NaN,Chlorophyll,60,62,63,80,80,60,ivysaur,base
2,Venusaur,Overgrow,NaN,Chlorophyll,80,82,83,100,100,80,venusaur,base
3,Mega Venusaur,Thick Fat,NaN,NaN,80,100,123,122,120,80,mega,venusaur
4,Charmander,Blaze,NaN,Solar Power,39,52,43,60,50,65,charmander,base


In [4]:
infinitedex = pd.read_csv("data/infinitefusiondex.csv")
infinitedex = infinitedex.rename(columns={'Species':'Name'})
infinitedex.head()

,DexId,Name,Type1,Type2
0,1,Bulbasaur,grass,poison
1,2,Ivysaur,grass,poison
2,3,Venusaur,grass,poison
3,4,Charmander,fire,fire
4,5,Charmeleon,fire,fire


In [5]:
infinitedex["Species"] = infinitedex["Name"].apply(lambda name: name.split()[0].lower())
# infinitedex["Form"] = infinitedex["Name"].apply(lambda name: name.split()[1].lower() if len(name.split()) > 1 else "base")
infinitedex.head()

,DexId,Name,Type1,Type2,Species
0,1,Bulbasaur,grass,poison,bulbasaur
1,2,Ivysaur,grass,poison,ivysaur
2,3,Venusaur,grass,poison,venusaur
3,4,Charmander,fire,fire,charmander
4,5,Charmeleon,fire,fire,charmeleon


In [16]:
completedex = pd.merge(
    infinitedex, 
    nationaldex[['Species','Form','ability_1','ability_2','ability_hidden','hp','attack','defense','sp_attack','sp_defense','speed']], 
    "inner",
    "Species"
    )
completedex = completedex.drop_duplicates(subset=["Name"])
completedex['total'] = completedex['hp'] + completedex['attack'] + completedex['defense'] + completedex['sp_attack'] + completedex['sp_defense'] + completedex['speed']
completedex = completedex.fillna("None")
completedex.head()

,DexId,Name,Type1,Type2,Species,Form,ability_1,ability_2,ability_hidden,hp,attack,defense,sp_attack,sp_defense,speed,total
0,1,Bulbasaur,grass,poison,bulbasaur,base,Overgrow,None,Chlorophyll,45,49,49,65,65,45,318
1,2,Ivysaur,grass,poison,ivysaur,base,Overgrow,None,Chlorophyll,60,62,63,80,80,60,405
2,3,Venusaur,grass,poison,venusaur,base,Overgrow,None,Chlorophyll,80,82,83,100,100,80,525
3,4,Charmander,fire,fire,charmander,base,Blaze,None,Solar Power,39,52,43,60,50,65,309
4,5,Charmeleon,fire,fire,charmeleon,base,Blaze,None,Solar Power,58,64,58,80,65,80,405


In [17]:
# turning all strings columns to lower case
completedex[completedex.select_dtypes(include='object').columns] = completedex.select_dtypes(include='object').apply(lambda x: x.str.lower())
completedex.head()

,DexId,Name,Type1,Type2,Species,Form,ability_1,ability_2,ability_hidden,hp,attack,defense,sp_attack,sp_defense,speed,total
0,1,bulbasaur,grass,poison,bulbasaur,base,overgrow,none,chlorophyll,45,49,49,65,65,45,318
1,2,ivysaur,grass,poison,ivysaur,base,overgrow,none,chlorophyll,60,62,63,80,80,60,405
2,3,venusaur,grass,poison,venusaur,base,overgrow,none,chlorophyll,80,82,83,100,100,80,525
3,4,charmander,fire,fire,charmander,base,blaze,none,solar power,39,52,43,60,50,65,309
4,5,charmeleon,fire,fire,charmeleon,base,blaze,none,solar power,58,64,58,80,65,80,405


In [18]:
target_species = ['dewgong','omanyte','omastar','scizor','empoleon','ferrothorn','celebi']
def swap_types(row):
    if row["Species"] in target_species:
        row["Type1"], row["Type2"] = row["Type2"], row["Type1"]
    return row

In [19]:
completedex = completedex.apply(lambda row:swap_types(row),axis=1)

In [20]:
completedex.to_csv("data/CompleteInfiniteFusionDex_v6.2.3.csv", index=False)